### AST 5731 
#### Group 1 - Project 1
#### Group Members: 

Style Notes (delete this cell prior to submission):
I figured we can use '###' for sections in markdown, then '####' if we want subsections (in which case we would probably do x.y formatting for the subsections). It might be tidier if we have markdown cells then code in each section, but we could also alternate between the two if people think that would be better

 ##### Contents
 1.   [Introduction](#introduction)
 2.   [Data](#data)
 3.   [Analysis](#analysis)   
    3.1 [Sensitivity Analysis](#senstivity)
 4.   [Results](#results)
 5.   [Discussion](#discussion)

### 1. Introduction 
<a id='introduction'></a>

Discussion of the problem and the goals of the project/ the questions we are trying to answer.

### 2. Data
<a id='data'></a>

Description of the data and how it was obtained (talk about TESS briefly I guess)

In [1]:
## Import libraries
import csv
import dis
import inspect
import os
import sys
import time


import astropy
import astroquery
import eleanor
import lightkurve as lk
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import scipy as sp
import scipy.stats as stats
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.time import TimeDelta
from astropy.timeseries import TimeSeries
from astropy.visualization import time_support
time_support()
from astropy.visualization import quantity_support
quantity_support()

from lightkurve import search_targetpixelfile

from scipy import linalg as la
from scipy import optimize
from scipy import integrate

from scipy.stats import norm
from scipy.stats import beta
from scipy.stats import binom

from IPython.display import display_html
from IPython.display import Image

%matplotlib inline

## set seed for reproducibility
random.seed(5731)

2022-09-29 13:11:38.131137: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-29 13:11:38.362217: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-29 13:11:38.373271: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-29 13:11:38.373290: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

### 3. Analysis
<a id='analysis'></a>

describe the model and what decisions we made in markdown, then do the analysis in python code cells

### 3.1 Sensitivity Analysis
<a id='sensitivity'></a>

### 4. Results
<a id='results'></a>

describe what we found and how we found it in markdown, then show plots and various things in python cells

### 5. Discussion
<a id='discussion'></a>

Maybe describe what we've learned? I'm not sure, this may be uneccessary